In [1]:
from flask import Flask
from flask_cors import CORS
from flask import request
import json
import pandas as pd
import pathlib
import heapq
import configparser

import requests
from datetime import timedelta
import base64
from urllib.parse import urlencode

from fastai.collab import *
from fastai.tabular.all import *

from flowsy.core import *

c:\users\daniel\anaconda3\envs\fastbook\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [11]:
config = configparser.ConfigParser()
config.read('secrets.ini')
client_id = config['secrets']['client_id']
client_secret = config['secrets']['client_secret']

In [12]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception('You must set client_id and client_secret')
        client_creds = f'{client_id}:{client_secret}'
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            'Authorization': f'Basic {client_creds_b64}'
        }
    
    def get_token_data(self):
        return {
            'grant_type': 'client_credentials'
        }
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            print(r)
            return False
        data = r.json()
        now = datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in']
        expires = now + timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def is_token_expired(self):
        return self.access_token_expires < datetime.now()

In [20]:
client = SpotifyAPI(client_id, client_secret)
client.perform_auth()

True

In [21]:
save = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath   
learn = load_learner('playlists_tracks_br_model_v1.pkl')
artist_learner = load_learner('playlists_artists_br_model_v1.pkl')
pathlib.PosixPath = save

# Required for version 1
track_factors = learn.model.i_weight.weight
playlist_factors = learn.model.u_weight.weight

dls = learn.dls

In [22]:
playlist_infos = json.load(open('playlist_infos_0-9999.json', 'r'))
playlists = json.load(open('minimized_mpd_0-9999.json', 'r'))
unique_tracks = json.load(open('unique_tracks_0-9999.json', 'r'))

playlists_csv = pd.read_csv('mpd_slice_0-9999.csv', delimiter=',', encoding='utf-8', header=None, low_memory=False,
                       names=['pid','track_uri','rating','playlist_name','track_name','artist_uri','artist_name'], skiprows=1)

In [23]:
def translate_track_uris(track_uris, n):
    tracks = []
    for track_uri in track_uris[:n]:
        for track in unique_tracks:
            if(track['track_uri'] == track_uri):
                tracks.append(track)
                break
    return tracks

In [ ]:
app = Flask("flowsy")
CORS(app)

@app.route('/token')
def getSpotifyToken():
    if(client.is_token_expired()):
        client.perform_auth()
    return json.dumps(client.access_token)

@app.route('/search/<filter>')
def search(filter):
    playlist_info = []
    for playlist in playlist_infos:
        if(filter.lower() in playlist['name'].lower()):
            playlist_info.append(playlist)
    return json.dumps(playlist_info)

@app.route('/playlist/<pid>')
def getPlaylistInfo(pid):
    return playlists[int(pid)]

@app.route('/tracks/<pid>')
def getTracksOfPlaylist(pid):
    return json.dumps(playlists[int(pid)]['tracks'])

@app.route('/recommendationsOfFlowsy/<pid>')
def getRecommendationsOfFlowsy(pid):
    
    rec = Recommender(p_learn=learn, a_learn=artist_learner, playlists=playlists_csv)
    rec_track_uris = rec.recommend(pid=int(pid))
    
    # Get number of requested tracks
    n = int(request.args.get('n'))
    
    return json.dumps(translate_track_uris(rec_track_uris, n))

@app.route('/recommendationsOfFlowsyv1/<pid>')
def getRecommendationsOfFlowsyv1(pid):
    
    idx = dls.classes['pid'].o2i[int(pid)]
    
    # Calculate dot products
    distances = np.array(list((playlist_factors[idx] @ track_factor for track_factor in track_factors)))
    
    # Get number of requested tracks
    n = int(request.args.get('n'))
    
    temp = n + playlists[int(pid)]['num_tracks']
    
    # Get tracks of highest dot product
    indices = heapq.nlargest(temp, range(len(distances)), distances.take)
    
    # Remove duplicates
    tracks_of_playlist = list(track['track_uri'] for track in playlists[int(pid)]['tracks'])
    top_track_uris = list((dls.classes['track_uri'][idx] for idx in indices))
    filtered_top_track_uris = [top_track_uri for top_track_uri in top_track_uris if not top_track_uri in tracks_of_playlist]
    
    # Translate recommended tracks
    top_tracks = []
    for track_uri in filtered_top_track_uris[:n]:
        for track in unique_tracks:
            if(track['track_uri'] == track_uri):
                top_tracks.append(track)
                break
    return json.dumps(top_tracks)

@app.route('/recommendationsOfJu/<pid>')
def getRecommendationsOfJu(pid):
    
    rec_track_uris = []
    
    # Get number of requested tracks
    n = int(request.args.get('n'))
    
    return json.dumps(translate_track_uris(rec_track_uris, n))
    
app.run(host='0.0.0.0')

 * Serving Flask app "flowsy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
10.0.0.4 - - [26/Mar/2021 19:19:48] "GET /playlist/52 HTTP/1.1" 200 -
10.0.0.4 - - [26/Mar/2021 19:19:48] "GET /tracks/52 HTTP/1.1" 200 -
10.0.0.4 - - [26/Mar/2021 19:19:48] "GET /token HTTP/1.1" 200 -
10.0.0.4 - - [26/Mar/2021 19:19:50] "GET /recommendationsOfFlowsy/52?n=50 HTTP/1.1" 200 -
10.0.0.4 - - [26/Mar/2021 19:21:05] "GET /playlist/52 HTTP/1.1" 200 -
10.0.0.4 - - [26/Mar/2021 19:21:05] "GET /tracks/52 HTTP/1.1" 200 -
10.0.0.4 - - [26/Mar/2021 19:21:05] "GET /token HTTP/1.1" 200 -
10.0.0.4 - - [26/Mar/2021 19:21:08] "GET /token HTTP/1.1" 200 -
10.0.0.4 - - [26/Mar/2021 19:21:08] "GET /search/null HTTP/1.1" 200 -
10.0.0.4 - - [26/Mar/2021 19:21:13] "GET /search/Christmas HTTP/1.1" 200 -
10.0.0.4 - - [26/Mar/2021 19:21:13] "GET /search/Christmas HTTP/1.1" 200 -
10.0.0.4 - - [26/Mar/2021 19:21:14] "GET /playlist/52 HTTP/1.1" 200 -
10.0.0.4 - - [26/Mar/2021 19:21:14] "GET /tracks/52 HTTP/1.1" 200 -
10.0.0.4 - - [26/Mar/2021 